Student Name : Sidong Mei             
Student Id Number : 45762295

# Assignment I Task-4

**Task-4** Anomaly Detection on two data sets (6 marks): 

Notes:

1). Predictive Maintenance Data: The data set tries to build an anomaly detection to predict and detect the machine failures. The dataset consists of 10000 data points stored as rows with 14 attributes in columns. You still need pre-processing and feature extraction for this data set. Particularly, the target is the attribute “Machine Failure” (0 for normal points and 1 for anomaly), and you could simple ignore the last five columns as they have been merged into the attribute “Machine Failure”. There are a several columns that might not be suitable to included. Note that the encoding of the target classes might affect the calculation of AUC scores.

2). Network Traffic Data: This is the data set used for a competition. The competition task was to build a network intrusion detector, a predictive model capable of distinguishing between “bad” connections, called intrusions or attacks, and “good” normal connections. This database contains a standard set of data to be audited, which includes a wide variety of intrusions simulated in a military network environment. To facilitate your data analysis process, we have done sampling and data pre-processing for you. All the features are named as “Feature #”, and the last attribute is the label. The label just has two values (1 for normal data and -1 for anomalies).

3). You are required to perform anomaly detection on the two data sets with multiple methods. One-class SVM is optional while we encourage you to complete it by learning it from the sklearn online document.  


In [1]:
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [2]:
data1=pd.read_csv('data/predictive-maintenance/predictive-maintenance.csv')
data1

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,0,0,0,0,0
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,0,0,0,0,0
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,0,0,0,0,0
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,0,0,0,0,0


## Data pre-processing

The data set consists of 13 features and a label. Totally 10000 rows and 14 columns. 

The columns are:

In [3]:
data1.columns

Index(['UDI', 'Product ID', 'Type', 'Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF',
       'RNF'],
      dtype='object')

The check the missing values and zero values in the data set.

In [4]:
print(data1.isna().sum().sort_values().tail(10))

Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64


In [5]:
(data1 == 0).astype(int).sum(axis=1).sort_values().tail(10)

1007    7
2245    7
6084    7
7256    7
4817    7
7762    7
2764    7
1510    7
250     7
0       7
dtype: int64

There are no missing values, check one rows with maximun number of 0 values and the 0 values is normal.

In [6]:
data1[data1.index==1007]

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
1007,1008,M15867,M,296.2,307.2,1633,31.1,0,0,0,0,0,0,0


In [7]:
(data1 == 0).astype(int).sum(axis=0).sort_values().tail(10)

Process temperature [K]       0
Rotational speed [rpm]        0
Torque [Nm]                   0
Tool wear [min]             120
Machine failure            9661
HDF                        9885
OSF                        9902
PWF                        9905
TWF                        9954
RNF                        9981
dtype: int64

Some column contains information which is not suitable for training. So, deleting those columns. Also delete last five rows.

In [8]:
data_clean=data1.drop(['UDI'], axis='columns')
data_clean.drop(['Product ID'], axis='columns',inplace=True)
data_clean.drop(index=[9995,9996,9997,9998,9999],inplace=True)

In [9]:
data_clean

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9990,L,298.8,308.5,1527,36.2,3,0,0,0,0,0,0
9991,M,298.9,308.4,1827,26.1,5,0,0,0,0,0,0
9992,L,298.8,308.4,1484,39.2,8,0,0,0,0,0,0
9993,L,298.8,308.4,1401,47.3,10,0,0,0,0,0,0


Now I get the clean data set. But the feature "Type" is categorical feature, it need to be transformed to integers array to make it can be used in next training steps. I use dictionary to transform the feature. 

In [10]:
data_clean['Type'].unique()

array(['M', 'L', 'H'], dtype=object)

In [11]:
dic = {'M':1,'L':2,'H':3}
data_clean['Type'] = data_clean['Type'].map(dic)

Finally, do the data normalization as the size range of each feature is inconsistent.

In [12]:
X = data_clean.drop(columns=['Machine failure'],axis=1)
anomaly_number = data_clean['Machine failure']
y=-1*(anomaly_number-1)

In [13]:
scaler = preprocessing.MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X = X_scaled
X

array([[0.        , 0.30434783, 0.35802469, ..., 0.        , 0.        ,
        0.        ],
       [0.5       , 0.31521739, 0.37037037, ..., 0.        , 0.        ,
        0.        ],
       [0.5       , 0.30434783, 0.34567901, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.5       , 0.38043478, 0.33333333, ..., 0.        , 0.        ,
        0.        ],
       [0.5       , 0.38043478, 0.33333333, ..., 0.        , 0.        ,
        0.        ],
       [0.5       , 0.38043478, 0.32098765, ..., 0.        , 0.        ,
        0.        ]])

Finally, I get a suitable data set for next training steps. 

## Anomaly Detection

First I calculate the anomaly rate in this data set.

In [14]:
print('\n data size: (%d, %d)\n' % data_clean.shape)
print('\n number of anomalies data:', anomaly_number.sum())
# Anomaly rate. Assuming that the target attribute has '1' for normal instances, and '0' for anomalies
print('\n Anomaly rate: %f \n' % (anomaly_number.sum()/X.shape[0]))


 data size: (9995, 12)


 number of anomalies data: 339

 Anomaly rate: 0.033917 



Then start training models. First, I choose Local Outlier Factor. This algorithm calculates Local Outlier Factor of each sample which measures local deviation of density of a given sample with respect to its neighbors. The sample with a substantially lower density than their neighbors will be recognized as outliers. 

Ref : https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.LocalOutlierFactor.html


Then I choose Area Under the Receiver Operating Characteristic Curve for evaluate the model performance. AUC is a value that can used to evaluate the two-category models. The AUC values closer to 1 means the performance is better, the more positive samples are predicted as positive, and the fewer negative samples are predicted as positive. 

Ref: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html


In [15]:
detector_lof = LocalOutlierFactor(n_neighbors=10, novelty=True,contamination=0.03)#contamination=0.03 because the anomaly rate is 0.03

# Train the model (Compute the LOF scores)
detector_lof.fit(X)

# Obtain anomaly score (Note that this is not the yes/no prediction as we don't know the threshold)
y_score = detector_lof.decision_function(X)
print(y_score)

[0.27602838 0.24358189 0.14858175 ... 0.24338776 0.27173916 0.20112402]


The decision_function of LOF model is opposite of the Local Outlier Factor, the bigger is normal. So, I transform labels and the 1 means normal and 0 means abnormal for calculating AUC score.

In [16]:
auc = roc_auc_score(y,y_score)
print('\n AUC score: %f\n' % auc)


 AUC score: 0.522988



In [17]:
y_pred = detector_lof.predict(X)
auc = roc_auc_score(y, y_pred)
print(y_pred)
print('\n AUC score: %f\n' % auc)

[1 1 1 ... 1 1 1]

 AUC score: 0.562408



 The performance of LOF is not very good as the AUC score is close to 0.5, the score is close to random sampling. 

The second model is Isolation Forest. It randomly selecting a feature and then randomly selecting a split value between the maximum and minimum values of the selected feature to make samples isolated. The process could be expressed as a tree. The abnormal sample will be in obvious shorter branches.

Ref: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html


In [41]:
detector_if = IsolationForest()

# Train the model
detector_if.fit(X)

# Obtain anomaly score (Note that this is not the yes/no prediction as we don't know the threshold)
y_score = detector_if.decision_function(X)
print(y_score)

[0.15143308 0.1466442  0.1432242  ... 0.14043863 0.15478576 0.11847173]


In [42]:
auc = roc_auc_score(y, y_score)
print('\n AUC score: %f\n' % auc)


 AUC score: 0.901987



In [43]:
y_pred = detector_if.predict(X)
print('\n AUC score: %f\n' % roc_auc_score(y, y_pred))


 AUC score: 0.765827



The result shows that the Isolation Forest performs better than LOF model. 

The third model is One class SVM. This algorithm will find a hyperplane that can divide normal data and abnormal data.

Ref：https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html


In [44]:
ocSVM = OneClassSVM(gamma='auto').fit(X)
y_pred=ocSVM.predict(X)
y_pred

array([ 1,  1, -1, ..., -1,  1,  1], dtype=int64)

In [45]:
auc = roc_auc_score(y, y_pred)
print('\n AUC score: %f\n' % auc)


 AUC score: 0.622277



The performance of One class SVM is better than LOF model but teerrible than isolation forest model.

### Conclusion 

In the first data set, the isolation forest model performs best, the auc score is 0.765827. The second best is one class SVM model, with auc score of 0.622277. And the worst model is LOF, with auc score of 0.562408. 

The difference of model performance is caused by the data. The data set one has 12 features, and majority of samples has 0 value on some of the features, so these two points will influence the performance of LOF and one class SVM models.

In [23]:
data2=pd.read_csv('data/network-traffic-analysis/kddcup99.csv')
data2

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,...,Feature 30,Feature 31,Feature 32,Feature 33,Feature 34,Feature 35,Feature 36,Feature 37,Feature 38,Label
0,0.000000,3.370000e-07,0.000049,0,0.0,0,0.0,0.0,1,0.0,...,0.858824,1.0,0.0,0.01,0.03,0.0,0.00,0.0,0.0,1
1,0.000000,3.480000e-07,0.000050,0,0.0,0,0.0,0.0,1,0.0,...,0.898039,1.0,0.0,0.01,0.03,0.0,0.00,0.0,0.0,1
2,0.000000,3.420000e-07,0.000328,0,0.0,0,0.0,0.0,1,0.0,...,1.000000,1.0,0.0,0.10,0.05,0.0,0.00,0.0,0.0,1
3,0.000000,4.280000e-07,0.000388,0,0.0,0,0.0,0.0,1,0.0,...,1.000000,1.0,0.0,0.00,0.01,0.0,0.00,0.0,0.0,1
4,0.000000,2.900000e-07,0.000664,0,0.0,0,0.0,0.0,1,0.0,...,1.000000,1.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6960,0.002057,4.020000e-07,0.000033,0,0.0,0,0.0,0.0,1,0.0,...,1.000000,1.0,0.0,0.03,0.07,0.0,0.00,0.0,0.0,1
6961,0.000000,3.970000e-07,0.000048,0,0.0,0,0.0,0.0,1,0.0,...,1.000000,1.0,0.0,0.50,0.08,0.0,0.00,0.0,0.0,1
6962,0.000000,4.700000e-07,0.000904,0,0.0,0,0.0,0.0,1,0.0,...,1.000000,1.0,0.0,0.04,0.10,0.0,0.00,0.0,0.0,1
6963,0.000000,3.490000e-07,0.001371,0,0.0,0,0.0,0.0,1,0.0,...,1.000000,1.0,0.0,0.02,0.05,0.0,0.00,0.0,0.0,1


The label has one strange value, so I delete the row.

In [24]:
data2['Label'].unique()

array(['1', '-1', 'f'], dtype=object)

In [25]:
data2.loc[data2['Label']=='f']

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,...,Feature 30,Feature 31,Feature 32,Feature 33,Feature 34,Feature 35,Feature 36,Feature 37,Feature 38,Label
4022,0.0,0.000001,0.00007,0,0.0,0,0.0,0.0,1,0.0,...,0.45098,0.45,0.08,0.0,0.0,0.02,0.02,0.0,0.0,f


In [26]:
data2.drop(index=[4022],inplace=True)

In [27]:
data2['Label'].unique()

array(['1', '-1'], dtype=object)

In [28]:
data2

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,...,Feature 30,Feature 31,Feature 32,Feature 33,Feature 34,Feature 35,Feature 36,Feature 37,Feature 38,Label
0,0.000000,3.370000e-07,0.000049,0,0.0,0,0.0,0.0,1,0.0,...,0.858824,1.0,0.0,0.01,0.03,0.0,0.00,0.0,0.0,1
1,0.000000,3.480000e-07,0.000050,0,0.0,0,0.0,0.0,1,0.0,...,0.898039,1.0,0.0,0.01,0.03,0.0,0.00,0.0,0.0,1
2,0.000000,3.420000e-07,0.000328,0,0.0,0,0.0,0.0,1,0.0,...,1.000000,1.0,0.0,0.10,0.05,0.0,0.00,0.0,0.0,1
3,0.000000,4.280000e-07,0.000388,0,0.0,0,0.0,0.0,1,0.0,...,1.000000,1.0,0.0,0.00,0.01,0.0,0.00,0.0,0.0,1
4,0.000000,2.900000e-07,0.000664,0,0.0,0,0.0,0.0,1,0.0,...,1.000000,1.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6960,0.002057,4.020000e-07,0.000033,0,0.0,0,0.0,0.0,1,0.0,...,1.000000,1.0,0.0,0.03,0.07,0.0,0.00,0.0,0.0,1
6961,0.000000,3.970000e-07,0.000048,0,0.0,0,0.0,0.0,1,0.0,...,1.000000,1.0,0.0,0.50,0.08,0.0,0.00,0.0,0.0,1
6962,0.000000,4.700000e-07,0.000904,0,0.0,0,0.0,0.0,1,0.0,...,1.000000,1.0,0.0,0.04,0.10,0.0,0.00,0.0,0.0,1
6963,0.000000,3.490000e-07,0.001371,0,0.0,0,0.0,0.0,1,0.0,...,1.000000,1.0,0.0,0.02,0.05,0.0,0.00,0.0,0.0,1


In [29]:
X = data2.drop(columns=['Label'],axis=1).values
y = data2['Label'].values

## Anomaly Detection

In [30]:
y=y.astype(np.int32)

I use LOF model, isolation forest model and one class SVM model to detect abnormal data in the second data set.

In [31]:
print('\n data size: (%d, %d)\n' % data2.shape)
print('\n number of anomalies data:', abs((y-1).sum()/2))
# Anomaly rate. Assuming that the target attribute has '1' for normal instances, and '0' for anomalies
print('\n Anomaly rate: %f \n' % (abs((y-1).sum()/2)/X.shape[0]))


 data size: (6964, 39)


 number of anomalies data: 246.0

 Anomaly rate: 0.035325 



In [32]:
detector_lof = LocalOutlierFactor(n_neighbors=10, novelty=True,contamination=0.03)

# Train the model (Compute the LOF scores)
detector_lof.fit(X)

# Obtain anomaly score (Note that this is not the yes/no prediction as we don't know the threshold)
y_score = detector_lof.decision_function(X)
print(y_score)

[2.50492992 0.7057063  2.68418254 ... 2.13077361 2.6598623  2.62724714]


In [33]:
auc = roc_auc_score(y,y_score)
print('\n AUC score: %f\n' % auc)


 AUC score: 0.635082



In [34]:
y_pred = detector_lof.predict(X)
auc = roc_auc_score(y, y_pred)
print(y_pred)
print('\n AUC score: %f\n' % auc)

[1 1 1 ... 1 1 1]

 AUC score: 0.524826



In [36]:
detector_if = IsolationForest()

# Train the model
detector_if.fit(X)

# Obtain anomaly score (Note that this is not the yes/no prediction as we don't know the threshold)
y_score = detector_if.decision_function(X)
print(y_score)

[0.15765407 0.15659987 0.15177129 ... 0.13921855 0.15381342 0.13609376]


In [37]:
auc = roc_auc_score(y, y_score)
print('\n AUC score: %f\n' % auc)


 AUC score: 0.907432



In [38]:
y_pred = detector_if.predict(X)
print('\n AUC score: %f\n' % roc_auc_score(y, y_pred))


 AUC score: 0.737629



In [39]:
ocSVM = OneClassSVM(gamma='auto').fit(X)
y_pred=ocSVM.predict(X)
y_pred

array([ 1,  1, -1, ..., -1,  1,  1], dtype=int64)

In [40]:
auc = roc_auc_score(y, y_pred)
print('\n AUC score: %f\n' % auc)


 AUC score: 0.622277



### Conclusion

On the second data set, the best model is still Isolation Forest, with the AUC score of 0.737629. The one class SVM and LOF model get the AUC score of 0.622277 and 0.524826. The second data set has 38 columns, affect performance of LOF and one class SVM model.

For classifier which use space distance (density), they may unsuitable for some data set who has many features. This tells me that it is important to select the model based on the data set. The distribution and type of data has a significant impact on the performance of the model.